In [19]:
import mysql.connector
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [97]:
bd = mysql.connector.connect(
    host= 'iosqlde.onairnet.xyz',
    user= 'vienna_airport',
    password= 'Vie23@#@',
    database= 'aeropuerto_viena',
    auth_plugin= 'mysql_native_password'
)

In [98]:
def sql(_query):
    return pd.read_sql_query(_query, bd)

In [63]:
#¿Cuantos litros se han repostado en cada zona operacional? Ordénalo de menor a mayor
sql('''SELECT count(id_gasolinera) as cantidad,sum(litros) as litros_totales,
CASE
	WHEN id_gasolinera=3 then 'ZONA B'
    WHEN id_gasolinera=6 then 'ZONA D'
    WHEN id_gasolinera=1 THEN 'ZONA A'
    WHEN id_gasolinera=2 THEN 'ZONA A'
    WHEN id_gasolinera=5 THEN 'ZONA C'
    WHEN id_gasolinera=4 THEN 'ZONA C'
    WHEN id_gasolinera=9 THEN 'ZONA E'
    WHEN id_gasolinera=7 THEN 'ZONA E'
    WHEN id_gasolinera=8 THEN 'ZONA E'
END AS zona_oper
FROM aeropuerto_viena.repostajes
group by zona_oper
order by litros_totales asc''')

,cantidad,litros_totales,zona_oper
0,378,10094.319983,ZONA E
1,406,10116.049991,ZONA C
2,392,10340.810000,ZONA D
3,411,10392.599996,ZONA B
4,416,10892.939974,ZONA A


In [77]:
#¿Cuántos vehículos han repostado más de 5L de RyanairHandling en la zona operacional A?¿Y en la E?
sql('''SELECT zonas_operacionales.nombre as Zonas,count(id_gasolinera) as cantidad,sum(repostajes.litros) as 
total_litros,handling.nombre as Operador
from vehiculos
inner join handling on id_handling=handling.id
inner join repostajes on id_vehiculo=vehiculos.id
inner join zonas_operacionales on vehiculos.id_zona=zonas_operacionales.id
where litros>5 and handling.nombre ='RYANAIR HANDLING' AND (zonas_operacionales.nombre LIKE '%A' OR zonas_operacionales.nombre LIKE '%E')
group by zonas_operacionales.nombre
order by total_litros desc''')

,Zonas,cantidad,total_litros,Operador
0,ZONA E,92,2484.470005,RYANAIR HANDLING
1,ZONA A,76,2203.579996,RYANAIR HANDLING


In [85]:
#¿Cuántos vehículos de media dan servicio a cada vuelo?
sql('''select total_vuelos,total as cantidad_veh,round(total/total_vuelos,2) as promedio_serv from(
select count(id_vuelo) as total_vuelos,sum(cantidad) as total from(
select id_vuelo,count(id_vehiculo) as cantidad FROM aeropuerto_viena.servicios
group by id_vuelo)sc)sc2''')

,total_vuelos,cantidad_veh,promedio_serv
0,3003,11968.0,3.99


In [89]:
#¿Cuál es el mayor operador de handling cada mes por cantidad de minutos de servicio prestados?

sql('''Select nombre,min_totales,
case
	when Mes=1 then'Enero'
    when Mes=2 then'Febrero'
    when Mes=3 then'Marzo'
    when Mes=4 then'Abril'
    when Mes=5 then'Mayo'
    when Mes=6 then'Junio'
    when Mes=7 then'Julio'
    when Mes=8 then'Agosto'
    when Mes=9 then'Septiembre'
    when Mes=10 then'Octubre'
    when Mes=11 then'Noviembre'
    when Mes=12 then'Diciembre'
end as mes
from(
select handling.nombre,
sum(minutos) as min_totales, 
month(vuelos.fecha) AS Mes FROM aeropuerto_viena.servicios
inner join aeropuerto_viena.vehiculos on id_vehiculo=vehiculos.id
inner join aeropuerto_viena.handling on vehiculos.id_handling=handling.id
inner join aeropuerto_viena.vuelos on servicios.id_vuelo=vuelos.id
group by handling.nombre,month(vuelos.fecha)
order by Mes,min_totales desc) sc''')

,nombre,min_totales,mes
0,VIENNA HANDLING,8022.0,Enero
1,RYANAIR HANDLING,7311.0,Enero
2,EASYHANDLING,6429.0,Enero
3,GROUND FORCE,5738.0,Enero
4,SWISSPORT,5273.0,Enero
5,VIENNA HANDLING,7458.0,Febrero
6,EASYHANDLING,6503.0,Febrero
7,RYANAIR HANDLING,6494.0,Febrero
8,GROUND FORCE,6021.0,Febrero
9,SWISSPORT,4793.0,Febrero


In [100]:
#Teniendo en cuenta que cada hora de servicio se factura a 60€, ¿Cuál es el beneficio por vehículo por cada empresa de handling?
#(Ten en cuenta que el combustible es un coste para ese vehículo)
sql('''SELECT sc3.nombre, sc3.tipo, round(sc3.beneficio - sc2.prec_repost_total,2) as beneficio_sin_repos
FROM (
    SELECT nombre, min_totales, tipo, ROUND((min_totales/60)*60,0) as beneficio
    FROM (
        SELECT handling.nombre, SUM(minutos) as min_totales, vehiculos.tipo
        FROM aeropuerto_viena.servicios
        INNER JOIN aeropuerto_viena.vehiculos ON id_vehiculo=vehiculos.id
        INNER JOIN aeropuerto_viena.handling ON vehiculos.id_handling=handling.id
        INNER JOIN aeropuerto_viena.vuelos ON servicios.id_vuelo=vuelos.id
        GROUP BY handling.nombre, tipo
        ORDER BY nombre
    ) sc
) sc3
LEFT JOIN (
    SELECT nombre, tipo, ROUND(SUM(litros*precio_litro),2) as prec_repost_total
    FROM aeropuerto_viena.repostajes
    INNER JOIN aeropuerto_viena.vehiculos ON repostajes.id_vehiculo=vehiculos.id
    INNER JOIN aeropuerto_viena.handling ON vehiculos.id_handling=handling.id
    GROUP BY nombre, tipo
    ORDER BY nombre
) sc2 ON sc3.nombre = sc2.nombre AND sc3.tipo = sc2.tipo''')

,nombre,tipo,beneficio_sin_repos
0,EASYHANDLING,Ambulift,5870.62
1,EASYHANDLING,Autobus Lanzadera,9964.61
2,EASYHANDLING,Camión de repostaje,4796.45
3,EASYHANDLING,Cinta Maletas,9814.11
4,EASYHANDLING,Coche de servicio,12736.70
5,EASYHANDLING,Escalera,7667.25
6,EASYHANDLING,Pushback,9375.87
7,EASYHANDLING,Tractor equipaje,11607.53
8,GROUND FORCE,Ambulift,8697.40
9,GROUND FORCE,Autobus Lanzadera,4296.52
